In [3]:
import pandas as pd 
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
#import pylab as plt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import statsmodels.api as sm
%matplotlib inline

###This notebook will focus on the analytical part of the 311 demographics analysis: that is, multilinear regressions between resident and working population socio-demographic attributes and  the number of 311 calls by type per capita, at the census tract level will be performed and analyzed

In [48]:
#Upload the working population attributes, and the resident population attributes at the CT level. Then, merging both datasets
demographics_NTA_NYC_residents=pd.read_csv('../datasets/version2/demographics_nta_NYC_residents.csv')
demographics_NTA_NYC_workers=pd.read_csv('../datasets/version2/demographics_nta_NYC_workers.csv')
demographics_NTA_NYC=pd.merge(demographics_NTA_NYC_residents,demographics_NTA_NYC_workers,on='Neighborhood',how='inner').drop('Unnamed: 0_y',axis=1)
print len(demographics_NTA_NYC_residents),len(demographics_NTA_NYC_workers),len(demographics_NTA_NYC)

195 194 194


In [49]:
residents_demographics=demographics_NTA_NYC_residents.columns[2:]
workers_demographics=demographics_NTA_NYC_workers.columns[2:]

In [50]:
#Upload the 311 calls by type per capita
calls_bytype_normalized=pd.read_csv('../datasets/version2/Call by type with normalization by resident - NTA level.csv').drop('Unnamed: 0',axis=1)

In [51]:
#callsbytype_attributes will be a dataframe combining all the information (demographics + calls by type)
callsbytype_attributes=pd.merge(calls_bytype_normalized,demographics_NTA_NYC, on='Neighborhood',how='inner')
print len(callsbytype_attributes), len(calls_bytype_normalized),len(demographics_NTA_NYC)

190 190 194


In [52]:
regressors=demographics_NTA_NYC.columns[2:]  #demographic indicators
types_of_calls=calls_bytype_normalized.columns[:-1]  #types of calls

Lets Consider different groups of population $g=1,2,…,n$ (based on our demographic indicators) and let:


$Pr(a,g)$ - the total number of residents in the location $a$ of group $g$ 

while $Pc(a,g)$ the number of commuters.
 
Let the unknown (subject to fit) complaining behavior be defined by the average number of complains of type $t$ per resident of group $g$ within his/her place of residency be $rc(g,t)$

Let also, $wc(g,t)$ be the number of complains of type $t$ per commuter of type $g$.

Then the total observed number of complains of type $t$ in the area $a$ is:

$$C(a,t)=\sum_{g,t} Pr(a,g) \ rc(g,t) + \sum_{g,t} Pc(a,g) \ wc(g,t) \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \  \text{(1)}$$ 

Then we know $Pr(a,g)$ and $Pc(a,g)$ (those are our regressors), we know the output variable $C(a,t)$ from 311 statistics. We need to fit the $rc(g,t)$, $wc(g,t)$ - slope coefficients of the multivariate linear regression.

This will give us complaining behavior per people of different groups and it will be distinguished by the complaining mode - while at home and while on the way.



###we will procced as follows:

STEP 1) Lasso regression:

Regressors:  

$Pr(a,g)$ - the total number of residents in the location $a$ of group $g$.
            
$Pc(a,g)$  number of commuters in the location a of group $g$

Target variable to be fit: $rc(g,t)$ -   average number of complains of type $t$ per resident of group $g$                                                      within his/her place of residency 

STEP 2) predict the number of complains per capita $wc(g,t)$ from the results of step 1, using equation $(1)$

Using the predicted value $rc(g,t)$ in each area, we are able to get a $wc(g,t)$ prediction (from the formula of the observed total calls by type $C(a,t)$ variable) 

In [53]:
results={}
for typeof in types_of_calls:
    A2=np.append(np.append(regressors,typeof),'Neighborhood')   #selection of columns
    myframe1=callsbytype_attributes[A2].dropna() 
    if len(myframe1)>50:
        results[typeof]={}
        X=myframe1[regressors]
        Y=myframe1[typeof]
        #LASSO
        X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
        X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
        R2_store=[]
        for i in range(-40,40):
            Lasso = linear_model.Lasso(fit_intercept=True,alpha=i)
            Lasso.fit(X_train,Y_train)
            R2_OS = 1 - np.var(Lasso.predict(X_val) - Y_val) / np.var(Y_val)
            R2_store.append(R2_OS)
        optim_alpha=range(-40,40)[np.where(R2_store==np.max(R2_store))[0][0]]
        results[typeof]['best_alpha']=optim_alpha
        Lasso = linear_model.Lasso(fit_intercept=True,alpha=optim_alpha)
        Lasso.fit(X_train,Y_train)
        R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
        results[typeof]['best_R2']=R2_OS
        results[typeof]['sample']=len(myframe1)

C:\Users\Marco\Anaconda\lib\site-packages\IPython\kernel\__main__.py:15: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
C:\Users\Marco\Anaconda\lib\site-packages\IPython\kernel\__main__.py:21: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


In [54]:
results_frame = pd.DataFrame(index=range(0,len(results.keys())), columns=['type','best_alpha','R2','sample_size'])

In [55]:
results_frame['type']=results.keys()
results_frame.head()
for i in results_frame.index:
    typeof=results_frame.loc[i,'type']
    results_frame.iloc[i,1]=results[typeof]['best_alpha']
    results_frame.iloc[i,2]=results[typeof]['best_R2']  
    results_frame.iloc[i,3]=results[typeof]['sample'] 

In [56]:
results_frame.sort('R2',ascending=False,inplace=True)

In [57]:
#save the progress so far
results_frame.to_csv('../outputs/Lasso_results_NTA.csv')

In [58]:
results_frame

,type,best_alpha,R2,sample_size
58,Sidewalk Condition,1,0.8376782,189
27,DOF Parking - Tax Exemption,1,0.8233698,58
89,Taxi Report,31,0.7796474,71
46,Food Poisoning,1,0.6916338,188
26,Air Quality,1,0.6694197,188
47,Noise,0,0.6110692,190
41,HEAT/HOT WATER,1,0.5377899,190
107,PAINT/PLASTER,1,0.5093141,188
38,Taxi Complaint,1,0.48521,184
69,APPLIANCE,1,0.4828439,187


Now, lets select the top complains and analyze the regression coefficients

In [59]:
top_R2=results_frame[results_frame.R2>=0.4]

In [60]:
top_list=np.array(top_R2['type'])

In [88]:
typeof=top_list[4]
A2=np.append(np.append(regressors,typeof),'Neighborhood')   #selection of columns
myframe1=callsbytype_attributes[A2].dropna() 
X=myframe1[regressors]
Y=myframe1[typeof]
#LASSO
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=1)
Lasso = linear_model.Lasso(fit_intercept=True,alpha=1)
Lasso.fit(X_train,Y_train)
R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
print R2_OS

0.793161985224


In [89]:
lasso_coefficients = pd.DataFrame(index=range(0,len(regressors)), columns=['regressor','coefficient'])

In [90]:
lasso_coefficients['regressor']=regressors
lasso_coefficients['coefficient']=Lasso.coef_

In [91]:
n=lasso_coefficients['coefficient']
thr=np.percentile(n, 90)
lasso_coefficients[np.abs(lasso_coefficients['coefficient'])>thr].sort('coefficient')

,regressor,coefficient
0,Total Population,-1.020444e-08
66,median house value,1.759930e-09
127,total workers,3.973033e-09
65,income per capita,4.349446e-09


In [92]:
typeof

'Air Quality'